In [ ]:
## https://machinelearningmastery.com
## /how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/

In [1]:
import pandas as pd
import numpy as np

In [2]:
# load all data
dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, \
                   infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])


In [3]:
dataset.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [ ]:
## It is a multivariate series comprised of seven variables (besides the date and time); they are:

## global_active_power: The total active power consumed by the household (kilowatts).
## global_reactive_power: The total reactive power consumed by the household (kilowatts).
## voltage: Average voltage (volts).
## global_intensity: Average current intensity (amps).
## sub_metering_1: Active energy for kitchen (watt-hours of active energy).
## sub_metering_2: Active energy for laundry (watt-hours of active energy).
## sub_metering_3: Active energy for climate control systems (watt-hours of active energy).

## Active and reactive energy refer to the technical details of alternative current.

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
Global_active_power      object
Global_reactive_power    object
Voltage                  object
Global_intensity         object
Sub_metering_1           object
Sub_metering_2           object
Sub_metering_3           float64
dtypes: float64(1), object(6)
memory usage: 126.7+ MB


In [5]:
print(len(dataset))
dataset.tail()

2075259


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2010-11-26 20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.0
2010-11-26 20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.0
2010-11-26 21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.0
2010-11-26 21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.0
2010-11-26 21:02:00,0.932,0.000,239.550,3.800,0.000,0.000,0.0


In [6]:
# mark all missing values indicated with a ‘?‘ character with a NaN value.

dataset.replace('?', 'nan', inplace=True)

# make dataset numeric

dataset = dataset.astype('float32')

print(len(dataset))
dataset.info()

2075259
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
Global_active_power      float32
Global_reactive_power    float32
Voltage                  float32
Global_intensity         float32
Sub_metering_1           float32
Sub_metering_2           float32
Sub_metering_3           float32
dtypes: float32(7)
memory usage: 71.2 MB


In [7]:
dataset.shape

(2075259, 7)

In [8]:
## We also need to fill in the missing values now that they have been marked.

## A very simple approach would be to copy the observation from the same time the day before. We can implement
## this in a function named fill_missing() that will take the NumPy array of the data and copy values from exactly
## 24 hours ago.

def fill_missing(values):
    one_day = 60 * 24
    for row in range(values.shape[0]):
        for col in range(values.shape[1]):
            if values[row, col] == 'nan':
                values[row, col] = values[row - one_day, col]

In [9]:
fill_missing(dataset.values)

In [10]:
dataset.values

array([[  4.216,   0.418, 234.84 , ...,   0.   ,   1.   ,  17.   ],
       [  5.36 ,   0.436, 233.63 , ...,   0.   ,   1.   ,  16.   ],
       [  5.374,   0.498, 233.29 , ...,   0.   ,   2.   ,  17.   ],
       ...,
       [  0.938,   0.   , 239.82 , ...,   0.   ,   0.   ,   0.   ],
       [  0.934,   0.   , 239.7  , ...,   0.   ,   0.   ,   0.   ],
       [  0.932,   0.   , 239.55 , ...,   0.   ,   0.   ,   0.   ]],
      dtype=float32)

In [11]:
dataset

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.400000,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.630005,23.000000,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.289993,23.000000,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.740005,23.000000,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.679993,15.800000,0.0,1.0,17.0
2006-12-16 17:29:00,3.520,0.522,235.020004,15.000000,0.0,2.0,17.0
2006-12-16 17:30:00,3.702,0.520,235.089996,15.800000,0.0,1.0,17.0
2006-12-16 17:31:00,3.700,0.520,235.220001,15.800000,0.0,1.0,17.0
2006-12-16 17:32:00,3.668,0.510,233.990005,15.800000,0.0,1.0,17.0


In [12]:
## A fourth sub-metering variable can be created by subtracting the sum of three defined sub-metering variables
## from the total active energy as follows:

values = dataset.values
dataset['sub_metering_4'] = (values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6])


In [13]:
dataset.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.4,0.0,1.0,17.0,52.266670
2006-12-16 17:25:00,5.360,0.436,233.630005,23.0,0.0,1.0,16.0,72.333336
2006-12-16 17:26:00,5.374,0.498,233.289993,23.0,0.0,2.0,17.0,70.566666
2006-12-16 17:27:00,5.388,0.502,233.740005,23.0,0.0,1.0,17.0,71.800003
2006-12-16 17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0,43.099998


In [14]:
# save updated dataset
dataset.to_csv('household_power_consumption_cleaned.csv')

In [17]:
## Problem Framing :
## There are many ways to harness and explore the household power consumption dataset.

## In this tutorial, we will use the data to explore a very specific question; that is:

## "Given recent power consumption, what is the expected power consumption for the week ahead?"

## This requires that a predictive model forecast the total active power for each day over the next seven days.

## Technically, this framing of the problem is referred to as a multi-step time series forecasting problem, given
## the multiple forecast steps. A model that makes use of multiple input variables may be referred to as a
## multivariate multi-step time series forecasting model.

## A model of this type could be helpful within the household in planning expenditures. It could also be helpful
## on the supply side for planning electricity demand for a specific household.

## This framing of the dataset also suggests that it would be useful to downsample the per-minute observations of
## power consumption to daily totals. This is not required, but makes sense, given that we are interested in total
## power per day.

## We can achieve this easily using the resample() function on the pandas DataFrame. Calling this function with the
## argument ‘D‘ allows the loaded data indexed by date-time to be grouped by day (see all offset aliases). We can 
## then calculate the sum of all observations for each day and create a new dataset of daily power consumption data
## for each of the eight variables.

# resample minute data to total for each day
# from pandas import read_csv
# load the new file
# dataset = read_csv('household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# resample data to daily
daily_groups = dataset.resample('D')
daily_data = daily_groups.sum()
# summarize
print(daily_data.shape)
#print(daily_data.head())
# save
daily_data.to_csv('household_power_consumption_days.csv')
daily_data.head()

(1442, 8)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-16,1209.176025,34.922001,93552.53125,5180.799805,0.0,546.0,4926.0,14680.933594
2006-12-17,3390.459961,226.005997,345725.31250,14398.599609,2033.0,4187.0,13341.0,36946.667969
2006-12-18,2203.825928,161.792007,347373.62500,9247.200195,1063.0,2621.0,14018.0,19028.433594
2006-12-19,1666.193970,150.942001,348479.00000,7094.000000,839.0,7602.0,6197.0,13131.900391
2006-12-20,2225.748047,160.998001,348923.62500,9313.000000,0.0,2648.0,14063.0,20384.800781


In [15]:
## Evaluation Metric
## A forecast will be comprised of seven values, one for each day of the week ahead.

## It is common with multi-step forecasting problems to evaluate each forecasted time step separately. This is
## helpful for a few reasons:

## To comment on the skill at a specific lead time (e.g. +1 day vs +3 days).
## To contrast models based on their skills at different lead times (e.g. models good at +1 day vs models good
## at days +5).
## The units of the total power are kilowatts and it would be useful to have an error metric that was also in the
## same units. Both Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) fit this bill, although RMSE is
## more commonly used and will be adopted in this tutorial. Unlike MAE, RMSE is more punishing of forecast errors.

## The performance metric for this problem will be the RMSE for each lead time from day 1 to day 7.

## As a short-cut, it may be useful to summarize the performance of a model using a single score in order to aide
## in model selection.

## One possible score that could be used would be the RMSE across all forecast days.

## The function evaluate_forecasts() below will implement this behavior and return the performance of a model
## based on multiple seven-day forecasts.

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

## Running the function will first return the overall RMSE regardless of day, then an array of RMSE scores for
## each day.

In [18]:
daily_data.tail(8)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2010-11-19,1570.400024,122.928001,345667.34375,6593.000000,0.0,483.0,11914.0,13776.333008
2010-11-20,2197.006104,153.768005,346476.00000,9320.200195,4367.0,2947.0,11433.0,17869.767578
2010-11-21,900.909973,119.624001,347299.46875,3798.600098,0.0,506.0,4778.0,9731.166992
2010-11-22,2041.536011,142.354004,345883.84375,8660.400391,4855.0,2110.0,10136.0,16924.599609
2010-11-23,1577.536011,137.449997,346428.75000,6731.200195,1871.0,458.0,7611.0,16352.266602
2010-11-24,1796.248047,132.460007,345644.59375,7559.399902,1096.0,2848.0,12224.0,13769.466797
2010-11-25,1431.163940,116.127998,347812.21875,6004.000000,1076.0,426.0,5072.0,17278.732422
2010-11-26,1488.104004,120.826004,303487.56250,6259.799805,1080.0,385.0,9989.0,13347.733398


In [14]:
daily_data[0:5]

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-16,1209.176025,34.922001,93552.53125,5180.799805,0.0,546.0,4926.0,14680.933594
2006-12-17,3390.459961,226.005997,345725.31250,14398.599609,2033.0,4187.0,13341.0,36946.667969
2006-12-18,2203.825928,161.792007,347373.62500,9247.200195,1063.0,2621.0,14018.0,19028.433594
2006-12-19,1666.193970,150.942001,348479.00000,7094.000000,839.0,7602.0,6197.0,13131.900391
2006-12-20,2225.748047,160.998001,348923.62500,9313.000000,0.0,2648.0,14063.0,20384.800781


In [19]:
from datetime import datetime

a='%Y-%m-%d'

#.zfill(x) fill up to x-1 with 0s so 1-1-1 will turn into 0001-1-1
#strptime gets only yyyy format, not yy, yy or y

stringdate1=input('Enter a 1st date in YYYY-MM-DD format :').zfill(8)
date1 = datetime.strptime(stringdate1, a)

stringdate2=input('Enter a 2nd date in YYYY-MM-DD format :').zfill(8)
date2 = datetime.strptime(stringdate2, a)

td=(date2-date1).days

#if not converting datetime into str with strftime - use .format

print('There are {} days between {} and {}'.format(
td, "{:%Y-%m-%d}".format(date1), "{:%Y-%m-%d}".format(date2)))



Enter a 1st date in YYYY-MM-DD format :2006-12-16
Enter a 2nd date in YYYY-MM-DD format :2006-12-20
There are 4 days between 2006-12-16 and 2006-12-20


In [20]:
dataset_days = pd.read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, \
                           parse_dates=['datetime'])

In [21]:
dataset_days.head()

,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
0,2006-12-16,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0,14680.934
1,2006-12-17,3390.460,226.006,345725.30,14398.6,2033.0,4187.0,13341.0,36946.668
2,2006-12-18,2203.826,161.792,347373.62,9247.2,1063.0,2621.0,14018.0,19028.434
3,2006-12-19,1666.194,150.942,348479.00,7094.0,839.0,7602.0,6197.0,13131.900
4,2006-12-20,2225.748,160.998,348923.62,9313.0,0.0,2648.0,14063.0,20384.800


In [22]:
dataset_days.tail(10)

,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
1432,2010-11-17,1582.032,141.916000,347691.22,6669.0,1153.0,491.0,8915.0,15808.200
1433,2010-11-18,1652.152,206.196000,347064.12,7022.8,2175.0,489.0,9449.0,15422.866
1434,2010-11-19,1570.400,122.928000,345667.34,6593.0,0.0,483.0,11914.0,13776.333
1435,2010-11-20,2197.006,153.768000,346476.00,9320.2,4367.0,2947.0,11433.0,17869.768
1436,2010-11-21,900.910,119.624000,347299.47,3798.6,0.0,506.0,4778.0,9731.167
1437,2010-11-22,2041.536,142.354000,345883.84,8660.4,4855.0,2110.0,10136.0,16924.600
1438,2010-11-23,1577.536,137.450000,346428.75,6731.2,1871.0,458.0,7611.0,16352.267
1439,2010-11-24,1796.248,132.460000,345644.60,7559.4,1096.0,2848.0,12224.0,13769.467
1440,2010-11-25,1431.164,116.128000,347812.22,6004.0,1076.0,426.0,5072.0,17278.732
1441,2010-11-26,1488.104,120.826004,303487.56,6259.8,1080.0,385.0,9989.0,13347.733


In [23]:
dataset_days_df = dataset_days.drop('datetime', axis = 1)
dataset_days_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
0,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0,14680.934
1,3390.460,226.006,345725.30,14398.6,2033.0,4187.0,13341.0,36946.668
2,2203.826,161.792,347373.62,9247.2,1063.0,2621.0,14018.0,19028.434
3,1666.194,150.942,348479.00,7094.0,839.0,7602.0,6197.0,13131.900
4,2225.748,160.998,348923.62,9313.0,0.0,2648.0,14063.0,20384.800


In [24]:
## Train and Test Sets
# We will use the first three years of data for training predictive models and the final year for evaluating models.

# The data in a given dataset will be divided into standard weeks. These are weeks that begin on a Sunday and end
# on a Saturday.

# The final year of the data is in 2010 and the first Sunday for 2010 was January 3rd. The data ends in mid November
# 2010 and the closest final Saturday in the data is November 20th. This gives 46 weeks of test data.

# The daily data starts in late 2006. The first Sunday in the dataset is December 17th, which is the second row of
# data.Organizing the data into standard weeks gives 159 full standard weeks for training a predictive model.

# The data ends in mid November 2010 and the closest final Saturday in the data is November 20th. 
####################################################################################################################
# Starting from 1st Sunday i.e. 2006-12-17 i.e. 2nd row of data-frame, total no. of days are :
td=((dataset_days['datetime'][len(dataset_days)-1]-dataset_days['datetime'][1]).days) + 1 # td = 1441 days
test_data_days = (3 * 52 + 3) * 7  ## = 1113, 3 weeks added to make it same as the example of this tutorial site.
balance_days = td - test_data_days  ## = 328
balance_weeks = balance_days // 7 ## = 46, '//' - this type of division used to get only the integer part of week
train_data_days = balance_weeks * 7 ## = 322

## split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    train = data[1:(test_data_days+1)]
    test = data[(test_data_days+1):-6]
    # restructure into windows of weekly data
    train = np.array(np.split(train, len(train)/7)).reshape(159,7,1)
    test = np.array(np.split(test, len(test)/7)).reshape(46,7,1)
    return train, test

## Actual splitting :
train, test = split_dataset(dataset_days_df['Global_active_power'])
# validate train data
print(train.shape)
print(train[0, 0, 0], train[-1, -1, 0])
# validate test
print(test.shape)
print(test[0, 0, 0], test[-1, -1, 0])

(159, 7, 1)
3390.46 1308.836
(46, 7, 1)
2083.4539999999997 2197.006


In [26]:
## split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks
    train = data[1:(test_data_days+1)]
    print(train.shape)
    print('=========================')
    
    test = data[(test_data_days+1):-6]
    print(test.shape)
    print('+++++++++++++++++++++++++++')
    
    # restructure into windows of weekly data
    train = np.array(np.split(train, len(train)/7)).reshape(159,7,8)
    test = np.array(np.split(test, len(test)/7)).reshape(46,7,8)
    return train, test

## Actual splitting :
train, test = split_dataset(dataset_days_df)
# validate train data
print(train.shape)
print(train[0, 0, 0], train[-1, -1, 0])
# validate test
print(test.shape)
print(test[0, 0, 0], test[-1, -1, 0])

(1113, 8)
(322, 8)
+++++++++++++++++++++++++++


ValueError: cannot copy sequence with size 7 to array axis with dimension 8

In [27]:
train = dataset_days_df[1:(test_data_days+1)]
print(len(train)/7)
aa=(np.split(train, 159))
print(type(aa))
print(type(train))
print(len(aa))

bb = np.empty(shape=(159,7,8))

for i in range(159):
    for j in range(7):
        for k in range(8):
            bb[i][j][k] = np.array(aa[i].iloc[j, k])
print(bb.shape)


bb[0]


159.0
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
159
(159, 7, 8)


array([[3.3904600e+03, 2.2600600e+02, 3.4572530e+05, 1.4398600e+04,
        2.0330000e+03, 4.1870000e+03, 1.3341000e+04, 3.6946668e+04],
       [2.2038260e+03, 1.6179200e+02, 3.4737362e+05, 9.2472000e+03,
        1.0630000e+03, 2.6210000e+03, 1.4018000e+04, 1.9028434e+04],
       [1.6661940e+03, 1.5094200e+02, 3.4847900e+05, 7.0940000e+03,
        8.3900000e+02, 7.6020000e+03, 6.1970000e+03, 1.3131900e+04],
       [2.2257480e+03, 1.6099800e+02, 3.4892362e+05, 9.3130000e+03,
        0.0000000e+00, 2.6480000e+03, 1.4063000e+04, 2.0384800e+04],
       [1.7166240e+03, 1.4416600e+02, 3.4661630e+05, 7.2386000e+03,
        1.7650000e+03, 2.6230000e+03, 1.0421000e+04, 1.3801400e+04],
       [2.3413380e+03, 1.8690600e+02, 3.4730575e+05, 9.8970000e+03,
        3.1510000e+03, 3.5000000e+02, 1.1131000e+04, 2.4390300e+04],
       [4.7733860e+03, 2.2147000e+02, 3.4579594e+05, 2.0200400e+04,
        2.6690000e+03, 4.2500000e+02, 1.4726000e+04, 6.1736434e+04]])

In [28]:
aa[0].iloc[:, 0]

1    3390.460
2    2203.826
3    1666.194
4    2225.748
5    1716.624
6    2341.338
7    4773.386
Name: Global_active_power, dtype: float64

In [ ]:
## Let us start a UNIVARIATE LSTM Model :

## https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [2]:
import numpy as np

In [3]:
univ_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [4]:
## split a univariate sequence into samples:

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
## returns X-numpy arrays of shape : no.of samples X n_steps and
## y-numpy arrays of shape : no.of samples X 

In [5]:
## Splitting above univariate sequence into samples of 3 time steps as input 'X' and one time step as output 'y' :

n_steps = 3
X, y = split_sequence(univ_seq, n_steps)


In [6]:

print(type(np.array(X)))
print(X.shape)

<class 'numpy.ndarray'>
(6, 3)


In [7]:
print('If 1st Input = ', X[0], '1st Output = ', y[0])
print('If 2nd Input = ', X[1], '2nd Output = ', y[1])
print('If 4th Input = ', X[3], '4th Output = ', y[3])
print('in_put shape = ', X.shape)
print('out_put shape = ', y.shape)

If 1st Input =  [10 20 30] 1st Output =  40
If 2nd Input =  [20 30 40] 2nd Output =  50
If 4th Input =  [40 50 60] 4th Output =  70
in_put shape =  (6, 3)
out_put shape =  (6,)


In [8]:
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])


[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [9]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

Using TensorFlow backend.


In [10]:
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from keras.layers import RepeatVector

In [11]:
## Vanila LSTM :

## We are working with a univariate series, so the number of features is one, for one variable.

# reshape input X from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) # try with activation = 'linear'
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# The shape of the input for each sample is specified in the input_shape argument on the definition of first
# hidden layer.

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[101.32057]]


In [13]:
## Vanila LSTM :

## We are working with a univariate series, so the number of features is one, for one variable.

# reshape input X from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=False, input_shape=(n_steps, n_features))) # try with activation = 'linear'
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# The shape of the input for each sample is specified in the input_shape argument on the definition of first
# hidden layer.

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[101.99585]]


In [ ]:
## https://machinelearningmastery.com/lstm-autoencoders/

In [14]:
## Stacked LSTM :

# define model
model = Sequential()
model.add(LSTM(50, activation='linear', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='linear'))  ## # try with activation = 'linear' at both places
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# An LSTM layer requires a three-dimensional input and LSTMs by default will produce a two-dimensional output
# as an interpretation from the end of the sequence.

# We can address this by having the LSTM output a value for each time step in the input data by setting the
# return_sequences=True argument on the layer. This allows us to have 3D output from hidden LSTM layer as input
# to the next.

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[102.05469]]


In [15]:
## Bidirectional LSTM :

# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[101.48527]]


In [26]:
## CNN LSTM :

# The first step is to split the input sequences into subsequences that can be processed by the CNN model. For
# example, we can first split our univariate time series data into input/output samples with four steps as input
# and one as output. Each sample can then be split into two sub-samples, each with two time steps. The CNN can
# interpret each subsequence of two time steps and provide a time series of interpretations of the subsequences
# to the LSTM model to process as input.

# We can parameterize this and define the number of subsequences as n_seq and the number of time steps per
# subsequence as n_steps. The input data can then be reshaped to have the required structure:

# choose a number of time steps
n_steps = 4

# split into samples
X, y = split_sequence(univ_seq, n_steps)

# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

# We want to reuse the same CNN model when reading in each sub-sequence of data separately.

# This can be achieved by wrapping the entire CNN model in a TimeDistributed wrapper that will apply the
# entire model once per input, in this case, once per input subsequence.

# The CNN model first has a convolutional layer for reading across the subsequence that requires a number of
# filters and a kernel size to be specified. The number of filters is the number of reads or interpretations of
# the input sequence. The kernel size is the number of time steps included of each ‘read’ operation of the input
# sequence.

# The convolution layer is followed by a max pooling layer that distills the filter maps down to 1/4 of their
# size that includes the most salient features. These structures are then flattened down to a single one-dimensional
# vector to be used as a single input time step to the LSTM layer.


# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), \
                          input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=500, verbose=0)

# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[100.57967]]


In [15]:
X.shape

(5, 2, 2, 1)

In [23]:
X[0]

array([[[10],
        [20]],

       [[30],
        [40]]])

In [27]:
## ConvLSTM :

# A type of LSTM related to the CNN-LSTM is the ConvLSTM, where the convolutional reading of input is built directly
# into each LSTM unit.

# The ConvLSTM was developed for reading two-dimensional spatial-temporal data, but can be adapted for use with
# univariate time series forecasting.

# The layer expects input as a sequence of two-dimensional images, therefore the shape of input data must be:

#                               [samples, timesteps, rows, columns, features]

# For our purposes, we can split each sample into subsequences where timesteps will become the number of
# subsequences, or n_seq, and columns will be the number of time steps for each subsequence, or n_steps. The number
# of rows is fixed at 1 as we are working with one-dimensional data :

# choose a number of time steps
n_steps = 4

# split into samples
X, y = split_sequence(univ_seq, n_steps)

# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

# We can define the ConvLSTM as a single layer in terms of the number of filters and a two-dimensional kernel size
# in terms of (rows, columns). As we are working with a one-dimensional series, the number of rows is always fixed
# to 1 in the kernel.

# The output of the model must then be flattened before it can be interpreted and a prediction made.

# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = np.array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.57239]]


In [28]:
X[0]

array([[[[10],
         [20]]],


       [[[30],
         [40]]]])

In [ ]:
## Multivariate LSTM Models

## There are two main models that we may require with multivariate time series data; they are:

## 1. Multiple Input Series.
## 2. Multiple Parallel Series.


In [29]:
## Multiple Input Series :

## A problem may have two or more parallel input time series and an output time series that is dependent on
## the input time series.

## The input time series are parallel because each series has an observation at the same time steps.

# define input & output sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# Convert above to a single dataset :

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)


[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [30]:
print(type(dataset))
print(dataset[0:3])
print('++++++++++++++')
print(dataset[0:3, :-1])
print('++++++++++++++')
print(dataset[2])
print('++++++++++++++')
print(dataset[2, -1])
print('++++++++++++++')
print(dataset[2, :-1])
print('++++++++++++++')


<class 'numpy.ndarray'>
[[10 15 25]
 [20 25 45]
 [30 35 65]]
++++++++++++++
[[10 15]
 [20 25]
 [30 35]]
++++++++++++++
[30 35 65]
++++++++++++++
65
++++++++++++++
[30 35]
++++++++++++++


In [31]:
# LSTMs can support parallel input time series as separate variables or features. Therefore, we need to split the
# data into samples maintaining the order of observations across the two input sequences.

# That is, if the first 'n' time steps of each parallel series are provided as input to the model the model should
# associate this with the value in the output series at the 'n-th' time step.

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [32]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
    
# We can see that the X component has a three-dimensional structure.

# The first dimension is the number of samples, in this case 7. The second dimension is the number of time steps
# per sample, in this case 3, the value specified to the function. Finally, the last dimension specifies the number
# of parallel time series or the number of variables, in this case 2 for the two parallel series.

# This is the exact three-dimensional structure expected by an LSTM as input. The data is ready to use without
# further reshaping.

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [33]:
X.shape[2]

2

In [34]:
## Vanilla LSTM :

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) # try with activation = 'linear'
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# The shape of the input for each sample is specified in the input_shape argument on the definition of first
# hidden layer.

# fit model
model.fit(X, y, epochs=200, verbose=0)

# demonstrate prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[208.05183]]


In [ ]:
## Multiple Parallel Series
# An alternate time series problem is the case where there are multiple parallel time series and a value must be
# predicted for each.

# For example, given the data from the previous section:
# [[ 10  15  25]
#  [ 20  25  45]
#  [ 30  35  65]
#  [ 40  45  85]
#  [ 50  55 105]
#  [ 60  65 125]
#  [ 70  75 145]
#  [ 80  85 165] 
#  [ 90  95 185]]
# We may want to predict the value for each of the three time series for the next time step. This might be
# referred to as multivariate forecasting.


In [35]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])


(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [36]:
## Vanilla LSTM
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[101.56405 108.04345 211.44759]]


In [37]:
## Stacked LSTM
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=400, verbose=0)
# demonstrate prediction
x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[102.14695 107.60262 209.53247]]


In [ ]:
## Multi-Step LSTM Models
# A time series forecasting problem that requires a prediction of multiple time steps into the future can be
# referred to as multi-step time series forecasting.

# Specifically, these are problems where the forecast horizon or interval is more than one time step.

# There are two main types of LSTM models that can be used for multi-step forecasting; they are:

#   1. Vector Output Model
#   2. Encoder-Decoder Model
#Before we look at these models, let’s first look at the preparation of data for multi-step forecasting.

In [38]:
# Data preparation :
# Here, both the input and output components will be comprised of multiple time steps and may or may not have
# the same number of steps.

# For example, given the univariate time series: [10, 20, 30, 40, 50, 60, 70, 80, 90] we could use 3 time steps as
# input and forecast the next 2 time steps.Here the 1st sample input is [10, 20, 30] and output is [40, 50]

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
print(X.shape)
print(y.shape)

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]
(5, 3)
(5, 2)


In [43]:
# Vector Output Model
# Like other types of neural network models, the LSTM can output a vector directly that can be interpreted as a
# multi-step forecast.

# This approach was seen in the previous section were one time step of each output time series was forecasted as a
# vector.

# As with the LSTMs for univariate data in a prior section, the prepared samples must first be reshaped. The LSTM
# expects data to have a three-dimensional structure of [samples, timesteps, features], and in this case, we only
# have one feature so the reshape is straightforward.

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0)
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[106.53106 122.68206]]


In [44]:
# univariate multi-step vector-output stacked lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='linear', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='linear'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0)
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[107.60564 125.18237]]


In [ ]:
# Encoder-Decoder Model
# A model specifically developed for forecasting variable length output sequences is called the Encoder-Decoder
# LSTM.

# The model was designed for prediction problems where there are both input and output sequences, so-called
# sequence-to-sequence, or seq2seq problems, such as translating text from one language to another.

# This model can be used for multi-step time series forecasting.

# As its name suggests, the model is comprised of two sub-models: the encoder and the decoder.

# (1) The encoder is a model responsible for reading and interpreting the input sequence. The output of the encoder
# is a fixed length vector that represents the model’s interpretation of the sequence. The encoder is traditionally
# a Vanilla LSTM model, although other encoder models can be used such as Stacked, Bidirectional, and CNN models.

# The decoder uses the output of the encoder as an input.

# (2) First, the fixed-length output of the encoder is repeated, once for each required time step in the output
# sequence.

# (3) This sequence is then provided to an LSTM decoder model. The model must output a value for each value in the
# output time step, which can be interpreted by a single output model.

# (4) We can use the same output layer or layers to make each one-step prediction in the output sequence. This can
# be achieved by wrapping the output part of the model in a TimeDistributed wrapper.

# (5) As with other LSTM models, the input data must be reshaped into the expected three-dimensional shape of
# [samples,timesteps, features].

# (6) In the case of the Encoder-Decoder model, the output, or y part, of the training dataset must also have this
# shape.
# This is because the model will predict a given number of time steps with a given number of features for each input
# sample.


In [45]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features] - Sl (5) & (6) above
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))  ## as per Sl. (5) above
y = y.reshape((y.shape[0], y.shape[1], n_features))  ## as per Sl. (6) above
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features))) ## encoder : as per Sl. (1) above
model.add(RepeatVector(n_steps_out))  ## as per Sl. (2) above
model.add(LSTM(100, activation='relu', return_sequences=True))  ## decoder : as per Sl. (3) above
model.add(TimeDistributed(Dense(1)))  ## output sequence as per Sl. (4) above
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=100, verbose=0)
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[[ 99.9752]
  [118.3034]]]


In [ ]:
## Multivariate Multi-Step LSTM Models
# In the previous sections, we have looked at univariate, multivariate, and multi-step time series forecasting.

# It is possible to mix and match the different types of LSTM models presented so far for the different problems.
# This too applies to time series forecasting problems that involve multivariate and multi-step forecasting, but
# it may be a little more challenging.

# In this section, we will provide short examples of data preparation and modeling for multivariate multi-step
# time series forecasting as a template to ease this challenge, specifically:

#      1. Multiple Input Multi-Step Output.
#      2. Multiple Parallel Input and Multi-Step Output.
# Perhaps the biggest stumbling block is in the preparation of data, so this is where we will focus our attention.

# 1. Multiple Input Multi-Step Output :
# There are those multivariate time series forecasting problems where the output series is separate but dependent
# upon the input time series, and multiple time steps are required for the output series.

# For example, consider our multivariate time series from a prior section:
#     [[ 10  15  25]
#      [ 20  25  45]
#      [ 30  35  65]
#      [ 40  45  85]
#      [ 50  55 105]
#      [ 60  65 125]
#      [ 70  75 145]
#      [ 80  85 165]
#      [ 90  95 185]]

# We may use three prior time steps of each of the two input time series to predict two time steps of the output
# time series.

# Input: 10, 15        Output: 65
#        20, 25                85
#        30, 35

In [71]:
# multivariate multi-step data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [72]:
## vector output with a Stacked LSTM model :

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[188.50548 211.53194]]


In [ ]:
# Multiple Parallel Input and Multi-Step Output:
# A problem with parallel time series may require the prediction of multiple time steps of each time series.

# For example, consider our multivariate time series from a prior section:
#    [[ 10  15  25]
#     [ 20  25  45]
#     [ 30  35  65]
#     [ 40  45  85]
#     [ 50  55 105]
#     [ 60  65 125]
#     [ 70  75 145]
#     [ 80  85 165]
#     [ 90  95 185]]

# We may use the last three time steps from each of the three time series as input to the model and predict the
# next time steps of each of the three time series as output.

# The first sample in the training dataset would be the following.

# Input: 10, 15, 25        Output: 40, 45, 85
#        20, 25, 45                50, 55, 105
#        30, 35, 65

In [73]:
# multivariate multi-step data preparation
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [74]:
## Encoder Decoder model :

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=0)
# demonstrate prediction
x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[ 92.19106   97.599976 190.41624 ]
  [103.94066  109.12895  213.57095 ]]]
